# Algoritmo de Bernstein-Vazirani — Implementación en Qiskit
## Microcredencial en Fundamentos de Computación Cuántica — Universidad de La Laguna

**Autor:** Hugo Tapia  
**Fecha:** Febrero 2026  
**Qiskit:** 2.x

---

**Objetivo:** Implementar el algoritmo de Bernstein-Vazirani de forma parametrizada, demostrando la ventaja cuántica $O(1)$ frente a la solución clásica $O(n)$.

**Estructura del notebook:**
1. Configuración del entorno
2. Solución clásica — $n$ consultas al oráculo
3. Implementación cuántica parametrizada
4. Ejemplo: $s = 11101101$ ($n = 8$ bits)

---
## 1. Configuración del entorno

In [ ]:
# Instalcion e importacion de paquetes :
 !pip install qiskit qiskit-aer pylatexenc

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

import qiskit
print(f"Qiskit version: {qiskit.__version__}")

---
## 2. Solución clásica

Para encontrar la cadena secreta $s$ de $n$ bits, la estrategia clásica requiere **$n$ consultas** al oráculo, una por cada bit. Cada consulta usa un vector canónico $e_i$ (un solo bit a 1) para aislar el bit $s_i$:

$$f_s(e_i) = s \cdot e_i \pmod{2} = s_i$$

In [ ]:
def oracle_clasico(s_secreta, x):
    """
    Oráculo clásico: calcula f_s(x) = s·x mod 2 (producto punto binario).
    """
    return sum(int(si) * int(xi) for si, xi in zip(s_secreta, x)) % 2


def solucion_clasica(s_secreta):
    """
    Descubre s consultando el oráculo n veces con vectores canónicos.
    """
    n = len(s_secreta)
    s_descubierta = ''
    
    print(f"Cadena secreta: s = {s_secreta} (n = {n} bits)")
    print(f"Se necesitan {n} consultas al oráculo:\n")
    
    for i in range(n):
        # Vector canónico e_i: solo el bit i está en 1
        x = '0' * i + '1' + '0' * (n - i - 1)
        bit_i = oracle_clasico(s_secreta, x)
        s_descubierta += str(bit_i)
        print(f"  Consulta {i+1}: x = {x}  →  f(x) = {bit_i}  →  s[{i}] = {bit_i}")
    
    print(f"\n✅ Cadena descubierta: s = {s_descubierta}")
    print(f"📊 Total consultas: {n}")
    return s_descubierta

In [ ]:
# Demostración clásica con s = '11101101' (n = 8 bits)
_ = solucion_clasica('11101101')

---
## 3. Implementación cuántica parametrizada

El algoritmo cuántico resuelve el mismo problema con **1 sola consulta** al oráculo, independientemente de $n$.

La función recibe cualquier cadena secreta $s$ de longitud arbitraria y construye automáticamente el circuito de tamaño $n$.

### 3.1 Oráculo cuántico

Para cada bit $s_i = 1$, se aplica una puerta CNOT entre el qubit $i$ y el auxiliar. Esto implementa $f_s(x) = s \cdot x \pmod{2}$ mediante el retroceso de fase (*phase kickback*).

In [ ]:
def crear_oraculo_bv(circuito, s_secreta, n):
    """
    Construye el oráculo de Bernstein-Vazirani.
    - Si s_i = 1: CNOT entre qubit i (control) y auxiliar (target)
    - Si s_i = 0: puerta identidad (claridad visual)
    
    Se invierte s para ajustar al orden little-endian de Qiskit.
    """
    s_invertida = s_secreta[::-1]
    for q in range(n):
        if s_invertida[q] == '1':
            circuito.cx(q, n)
        else:
            circuito.id(q)

### 3.2 Algoritmo completo parametrizado

In [ ]:
def bernstein_vazirani(s_secreta, shots=1024):
    """
    Algoritmo de Bernstein-Vazirani parametrizado.
    El tamaño n se determina automáticamente de la cadena secreta.
    
    Args:
        s_secreta (str): Cadena de bits a descubrir (ej: '011', '11101101')
        shots (int): Número de ejecuciones del circuito
    Returns:
        dict: Conteo de mediciones
        QuantumCircuit: Circuito construido
    """
    n = len(s_secreta)
    
    # PASO 1: Crear circuito — n qubits entrada + 1 auxiliar + n bits clásicos
    bv_circuit = QuantumCircuit(n + 1, n)
    
    # PASO 2: Preparar auxiliar en |−⟩ (con H + Z)
    bv_circuit.h(n)
    bv_circuit.z(n)
    
    # PASO 3: Hadamard en registro de entrada → superposición
    for i in range(n):
        bv_circuit.h(i)
    
    bv_circuit.barrier()
    
    # PASO 4: Oráculo
    crear_oraculo_bv(bv_circuit, s_secreta, n)
    
    bv_circuit.barrier()
    
    # PASO 5: Hadamard de nuevo → interferencia
    for i in range(n):
        bv_circuit.h(i)
    
    # PASO 6: Medir
    for i in range(n):
        bv_circuit.measure(i, i)
    
    # Ejecutar en simulador
    simulador = AerSimulator()
    circuito_compilado = transpile(bv_circuit, simulador)
    resultado = simulador.run(circuito_compilado, shots=shots).result()
    conteos = resultado.get_counts()
    
    # Verificar
    estado_medido = max(conteos, key=conteos.get)
    print(f"Cadena secreta: s = {s_secreta} (n = {n})")
    print(f"Resultado medido: {estado_medido} — {'✅ Correcto' if estado_medido == s_secreta else '❌ Incorrecto'}")
    print(f"Consultas al oráculo: 1 (clásico necesitaría {n})")
    
    return conteos, bv_circuit

---
## 4. Ejemplo: $s = 11101101$ ($n = 8$ bits)

### 4.1 Ejecución del algoritmo

In [ ]:
conteos, circuito = bernstein_vazirani('11101101')

### 4.2 Visualización del circuito

In [ ]:
circuito.draw('mpl', style='iqp')

### 4.3 Histograma de resultados

In [ ]:
plot_histogram(conteos, title='Bernstein-Vazirani: s = 11101101 (n = 8)')

---
### Resultado

El simulador devuelve `11101101` en el 100% de las 1024 ejecuciones, confirmando que el algoritmo cuántico encuentra la cadena secreta de 8 bits con **una sola consulta** al oráculo frente a las 8 que necesita el método clásico.